# SHARK ATTACK:  a data cleaning and manipulation project.

## Import Libraries 

In [182]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import re

## Data Reading

In [183]:
data = pd.read_csv("GSAF5.csv", engine="python", sep=',', quotechar='"', error_bad_lines=False)

We'll check out the first ROWs to familiarize ourselves with the table

In [184]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


We'll also .shape it to see it's size.

In [185]:
data.shape

(5992, 24)

## Header Cleaning

We print the column names, not only to get get further knowledge of what we have in the table, but to see if we have to modify the names.

In [186]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

We see some names have extra spaces (`'Sex '`, `'Species '`), non-alphanumeric (`'Fatal (Y/N)'`, `'Case Number.1'`, `'Case Number.2'`, `'Unnamed: 22'`, `'Unnamed: 23'`]) or don't start with capital letter (`'pdf'`, `'href formula'`, `'href'`, `'original order'`). We will make sure all of them follow the same style: Xxxx.

In [187]:
data.columns = ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Species', 'Investigator or Source', 'Pdf', 'Href Formula', 'Href',
       'Case Number 1', 'Case Number 2', 'Original Order', 'Unnamed 22',
       'Unnamed 23']

We print the column names again to make sure that the name changing worked.

In [188]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'Pdf', 'Href Formula', 'Href',
       'Case Number 1', 'Case Number 2', 'Original Order', 'Unnamed 22',
       'Unnamed 23'],
      dtype='object')

## Header Organization

Once we have all the column names in the same style we will organize them; in this case I've choosen to group them by similar content

In [189]:
column_order = ['Case Number', 'Case Number 1','Case Number 2', 'Date', 'Year', 'Time',  'Country', 'Area', 
                'Location','Name', 'Sex', 'Age', 'Activity', 'Type', 'Species', 'Injury', 'Fatal', 
                'Investigator or Source', 'Pdf', 'Href Formula', 'Href','Original Order', 'Unnamed 22',
                'Unnamed 23']
data = data[column_order]

We print the column names again to make sure that the reorganization worked.

In [190]:
data.columns

Index(['Case Number', 'Case Number 1', 'Case Number 2', 'Date', 'Year', 'Time',
       'Country', 'Area', 'Location', 'Name', 'Sex', 'Age', 'Activity', 'Type',
       'Species', 'Injury', 'Fatal', 'Investigator or Source', 'Pdf',
       'Href Formula', 'Href', 'Original Order', 'Unnamed 22', 'Unnamed 23'],
      dtype='object')

## NULLs

When you delete Data you have to be careful and very sure of what you're doing, that is why for the first filter we will check the percentages of NULLs per columns

In [191]:
data.isnull().sum()* 100 / len(data)

Case Number                0.000000
Case Number 1              0.000000
Case Number 2              0.000000
Date                       0.000000
Year                       0.000000
Time                      53.621495
Country                    0.717623
Area                       6.708945
Location                   8.277704
Name                       3.337784
Sex                        9.462617
Age                       44.742991
Activity                   8.795060
Type                       0.000000
Species                   48.965287
Injury                     0.450601
Fatal                      0.317089
Investigator or Source     0.250334
Pdf                        0.000000
Href Formula               0.016689
Href                       0.050067
Original Order             0.000000
Unnamed 22                99.983311
Unnamed 23                99.966622
dtype: float64

We see that the following columns are the ones that contain over a 40% NULLS and have no complementary column from where we can extract information to complete them:
<br><br>`Age`: 44.74%
<br><br>`Species`: 48.96%
<br><br>`Time`: 53.62%
<br><br>`Unnamed: 22`: 99.98%
<br><br>`Unnamed: 23`: 99.96%

The following function DROPs all the columns that have more than a 40% NULL (all of the above)

In [192]:
def remove_null_columns(df):
    dff = pd.DataFrame()
    for cl in df:
        if df[cl].isnull().sum()* 100 / len(data) >= 40:
            pass
        else:
            dff[cl] = df[cl]
    return dff  
data = remove_null_columns(data)

We will return the columns that we're left with

In [193]:
data.columns

Index(['Case Number', 'Case Number 1', 'Case Number 2', 'Date', 'Year',
       'Country', 'Area', 'Location', 'Name', 'Sex', 'Activity', 'Type',
       'Injury', 'Fatal', 'Investigator or Source', 'Pdf', 'Href Formula',
       'Href', 'Original Order'],
      dtype='object')

We will check what NULLs we have left to see how to continue.

In [194]:
data.isnull().sum()* 100 / len(data)

Case Number               0.000000
Case Number 1             0.000000
Case Number 2             0.000000
Date                      0.000000
Year                      0.000000
Country                   0.717623
Area                      6.708945
Location                  8.277704
Name                      3.337784
Sex                       9.462617
Activity                  8.795060
Type                      0.000000
Injury                    0.450601
Fatal                     0.317089
Investigator or Source    0.250334
Pdf                       0.000000
Href Formula              0.016689
Href                      0.050067
Original Order            0.000000
dtype: float64

We can see some columns with a degree of connection between them an a small percentage of NULL; further on we will try to complete the columns that interest us the most:
<br>
<br>Related columns with a small percentage of NULLs:<br>
<br>**GROUP 1**
<br>`Country`: 0.7%
<br>`Area`: 6.7%
<br>`Location`: 8.2%
<br>We will focus on `Country` given that is the one with the smaller percentage of NULLs<br>
<br>**GROUP 2**
<br>`Injury`: 0.4%
<br>`Fatal`: 0.3%
<br>We will focus on `Fatal` given that, although minimal difference, it has a lower percentage of NULLs<br>
<br>**GROUP 3**
<br>`Href Formula`: 0.01%
<br>`Href`: 0.05%
<br>We will focus on `Href` given that, although minimal difference, it has a lower percentage of NULLs

## Row Cleaning

We have to consider that we might also have NULLs in various positions in a same ROW, that is why, when we start the coulmn cleaning we will be filtering by various columns and droping rows. For that. we will create a function that drop the rows.

In [195]:
def drop_rows(x):
    index_drop = [index for index in x.index]
    index_drop.reverse()
    for i in index_drop:
        data.drop(data.index[i], inplace = True)

## Column cleaning

We will try to complete the columns that have NULLs with the ones that have complementary information or DROP the ROWS that don't have enough information to be usefull.<br>
<br>For that we will follow the subsequent steps:
<br>
<br>1. Selecte the columns we will be filtering<br>
<br>2. Filter all of them with .isnull()<br>
<br>3. Filter all of them except one with .notnull(), the selected column can not be the choosen to complete. If there are more than two columns selected repeat with respectevely<br>
<br>4. Once you have the information you need for the column (or you don't)

    Traspass the information
    Drop the ROWs

<br>5. Drop the unneeded columns<br>

<br>**NOTE: Every time you drop rows check the shape of the table to make sure they have been properly eliminated**


### GROUP 1: Location, Area & Country

We will focus on `Country` so we will be droping all of those ROW where we can't exctract a Country Name from `Area` or `Location`. <br>
<br>To double check the information we will start by checking how many NULLs `Country` has.

In [196]:
data["Country"].isnull().sum()

43

Using the function we will DROP all the ROWS where `Location`, `Area`, and `Country` is NULL

In [197]:
data[(data["Location"].isnull()) & (data["Area"].isnull()) & (data["Country"].isnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Area,Location,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
303,2014.08.00,2014.08.00,2014.08.00,Aug-14,2014,NaN,NaN,NaN,Cuban refugees,M,Sea disaster,Invalid,Shark involvement prior to death not confirmed,Y,"Associated Press, 11/27/2014",2014.08.00-Cuban-refugees.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5689
3153,1970.11.00,1970.11.00,1970.11.00,Nov-70,1970,NaN,NaN,NaN,Heinz Plotsky,M,NaN,Unprovoked,Extensive injuries,N,"H.D. Baldridge (1994), SAF Case #1645",1970.11.00-NV-Plotsky.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2840
3163,1970.07.05,1970.07.05,1970.07.05,05-Jul-70,1970,NaN,NaN,NaN,male,M,NaN,Unprovoked,Finger or toe severed,N,"H.D. Baldridge (1994), SAF Case #1628",1970.07.05-NV-male.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2830
3170,1970.04.00.b,1970.04.00.b,1970.04.00.b,Apr-70,1970,NaN,NaN,NaN,Lionel Jarvis,M,Freediving,Provoked,Arm abraded & lacerated. Recorded as PROVOKED ...,N,"H.D. Baldridge (1994), SAF Case #1616",1970.04.00.b-NV-Jarvis.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2823
3174,1970.02.05,1970.02.05,1970.02.05,05-Feb-70,1970,NaN,NaN,NaN,Sally Anne Irvine,F,Wading,Unprovoked,Lacerations to lower leg,N,H.D. Baldridge (1994) SAF Case #1626,1970.02.05-NV-Irvine.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2819
3200,1969.08.00,1969.08.00,1969.08.00,Aug-69,1969,NaN,NaN,NaN,Rodney Hughes,M,NaN,Unprovoked,Am lacerated,N,H.D. Baldridge (1994) SAF Case #1602,1969.08.00-NV-Hughes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2793
3435,1964.09.27,1964.09.27,1964.09.27,27-Sep-64,1964,NaN,NaN,NaN,Giancarlo Griffon,M,Spearfishing,Invalid,"Disappeared, probable drowning but sharks in a...",NaN,C. Moore. GSAF,1964.09.27-Griffon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2558
4499,1942.00.00.e,1942.00.00.e,1942.00.00.e,1942,1942,NaN,NaN,NaN,male,M,Jumped overboard from torpedoed Panamanian fre...,Sea Disaster,FATAL,Y,"V.M. Coppleson (1962), p.258",1942.00.00.e-seaman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1494
4502,1942.00.00.b,1942.00.00.b,1942.00.00.b,1942,1942,NaN,NaN,NaN,"Bombardier J. Hall, Private Green of the Sherw...",M,"Days before the surrender of Singapore, the 3 ...",Boating,No injury to occupants. Sharks continually fol...,N,"V.M. Coppleson (1962), p.206",1942.00.00.b-Hall-Green-Jennings.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1491
4681,1935.04.12.R,1935.04.12.R,1935.04.12.R,Reported 12-Apr-1935,1935,NaN,NaN,NaN,Pearl Purdy Scott,F,Diving,Unprovoked,Laceration to left leg,N,"Port Arthur News, 4/12/1935",1935.04.12.R-PearlPurdyScott.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1312


In [198]:
filter_by_country = data[(data["Location"].isnull()) & (data["Area"].isnull()) & (data["Country"].isnull())]
drop_rows(filter_by_country)

Double check how many NULLs `Country` has to make sure the function works, we will be doing this every time we pass the function.

In [199]:
data["Country"].isnull().sum()

19

FILTER by `Location`: we will check the ROWs where `Location` is not NULL

In [200]:
data[(data["Location"].notnull()) & (data["Area"].isnull()) & (data["Country"].isnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Area,Location,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
3379,1965.10.21,1965.10.21,1965.10.21,21-Oct-65,1965,NaN,NaN,Florida Strait,Mario Castellanos,M,The boat Caribou II sank,Unprovoked,Survived,N,"Lodi News Sentinel, 10/30/1965",1965.10.21-Castellanos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2614
4412,1944.10.24,1944.10.24,1944.10.24,24-Oct-44,1944,NaN,NaN,225 miles east of Hong Kong,NaN,M,Japanese POW ship Arisan Maru with 1800 Americ...,Sea Disaster,Most of the men drowned & some were taken by s...,Y,internet (multiple),1944.10.24-ArisanMaru.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1581
5189,1909.01.17,1909.01.17,1909.01.17,17-Jan-09,1909,NaN,NaN,Near the equator,Thomas Butler,M,Jumped overboard,Invalid,FATAL,Y,"Star, 3/18/1909",1909.01.17-Butler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,804
5560,1881.06.25,1881.06.25,1881.06.25,25-Jut-1881,1881,NaN,NaN,Santa Cruz,Father Hudson,M,Bathing,Unprovoked,Survived,N,"Grey River Argus,10/3/1881, p.2",1881.06.25-FatherHudson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,433
5847,1742.12.17,1742.12.17,1742.12.17,17-Dec-1742,1742,NaN,NaN,Carlisle Bay,2 impressed seamen,M,Swimming,Unprovoked,FATAL,Y,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,146
5896,nd-0114,nd-0114,nd-0114,Before 2012,0,NaN,NaN,In a river feeding into the Bay of Bengal,Sametra Mestri,F,Netting shrimp,Unprovoked,Hand severed,N,National Georgraphic Television,ND-0114-BayOfBengal.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,97


In [201]:
filter_by_location = data[(data["Location"].notnull()) & (data["Area"].isnull()) & (data["Country"].isnull())]

We will DROP this ROWs given that the information in them is:
 
1) not usesful
 
2) not reliable

In [202]:
drop_rows(filter_by_location)

Double check how many NULLs Country has to make sure the function works.

In [203]:
data["Country"].isnull().sum()

19

THIRD: FILTER by `Area`: we will check the ROWs where `Area` is not NULL

In [204]:
data[(data["Location"].isnull()) & (data["Area"].notnull()) & (data["Country"].isnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Area,Location,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
2731,1983.00.00.d,1983.00.00.d,1983.00.00.d,Ca. 1983,1983,NaN,English Channel,NaN,Padma Shri Taranath Narayan Shenoy,M,Swimming,Unprovoked,Left leg bitten,N,"Times of India, 2/5/2012",1983.00.00.d-Shenoy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3262
3792,1960.01.26,1960.01.26,1960.01.26,26-Jan-60,1960,NaN,"Between Timor & Darwin, Australia",NaN,NaN,NaN,Portuguese Airliner with 9 people aboard went ...,Sea Disaster,"As searchers approached wreckage, sharks circl...",N,"V.M. Coppleson (1962), p.260",1960.01.26-Portuguese airliner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2201
4005,1956.09.13,1956.09.13,1956.09.13,13-Sep-56,1956,NaN,Near the Andaman & Nicobar Islands,NaN,male,M,Climbing back on ship,Unprovoked,FATAL,Y,M. Hosina,1956.09.13-TunaBoat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1988
4473,1942.11.00.a,1942.11.00.a,1942.11.00.a,Nov-42,1942,NaN,Off South American coast,NaN,NaN,M,Dutch merchant ship Zaandam torpedoed by the ...,Sea Disaster,FATAL,Y,"M. Murphy; V.M. Coppleson (1962), pp.207-208",1942.11.00.a-Izzi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1520
4485,1942.06.00,1942.06.00,1942.06.00,Jun-42,1942,NaN,300 miles east of St. Thomas (Virgin Islands),NaN,male,M,On life raft tethered to lifeboat. A seaman pu...,Unprovoked,Forearm lacerated,N,"V.M. Coppleson (1962), p.258",1942.06.00-on-life-raft.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1508
5370,1897.03.15.b.R,1897.03.15.b.R,1897.03.15.b.R,Reported 15-Mar-1897,1897,NaN,Mediterranean Sea,NaN,male,M,Swimming,Unprovoked,FATAL,Y,"Daily Northwestern, 5/15/1897",1897.03.15.b.R-Mediterranean.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,623
5558,1881.08.16.R,1881.08.16.R,1881.08.16.R,Reported 16-Aug-1881,1881,NaN,Western Banks,NaN,George Sedgwick,M,"Floating, holding onto an oar after dory capsized",Unprovoked,FATAL,Y,"Lewiston Evening Journal, 8/16/1881",1881.08.16.R-Sedgwick.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,435
5866,0077.00.00,0077.00.00,0077.00.00,77 A.D.,77,NaN,Ionian Sea,NaN,males,M,Sponge diving,Unprovoked,FATAL,Y,Perils mentioned by Pliny the Elder (23 A.D. t...,77AD-Pliny.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,127
5868,0.0214,0.0214,0.0214,Ca. 214 B.C.,0,NaN,Ionian Sea,NaN,"Tharsys, a sponge diver",M,Ascending from a dive,Unprovoked,"FATAL, shark/s bit him in two",Y,"Reported by Greek poet, Leonidas of Tarentum (...",214BC-Tharsys.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,125


We will DROP this ROWS given that the Areas are very big.

In [205]:
filter_by_area = data[(data["Location"].isnull()) & (data["Area"].notnull()) & (data["Country"].isnull())]
drop_rows(filter_by_area)

Double check how many NULLs `Country` has to make sure the function works.

In [206]:
data["Country"].isnull().sum()
data["Country"].isnull().sum()* 100 / len(data)

18

0.30236855367041826

We can see we have reduced from 43 to 5 NULL ROWs in `Country` and once that the corresponding ROWS have been DROP we can start to complete the missing information.

To complete the missing information we will have to extract it from `Location` and `Area` to traspass it to `Country`.

In [207]:
data[(data["Location"].notnull()) & (data["Area"].notnull()) & (data["Country"].isnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Area,Location,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
3162,1970.08.02,1970.08.02,1970.08.02,02-Aug-70,1970,NaN,Caribbean Sea,Between St. Kitts & Nevis,NaN,NaN,Sea Disaster Sinking of ferryboat Christina,Invalid,"Sharks scavenged on bodies, but no record of t...",Y,"Rome News Tribune, 8/3/1970",1970.08.02-Christina-ferryboat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2831
4040,1956.00.00.g,1956.00.00.g,1956.00.00.g,1956,1956,NaN,Between Comores & Madagascar,Geyser Bank,"Captain Eric Hunt, the cook & a French passenger",M,Shipwreck,Sea Disaster,FATAL,Y,dinofish.com,1956.00.00.g-Capt-Hunt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1953
4271,1949.12.00.b,1949.12.00.b,1949.12.00.b,Dec-49,1949,NaN,Caribbean Sea,Between Cuba & Costa Rica,"Albert Battles, James Dean & 4 crew",M,"Sea Disaster, sinking of the motorship Wingate",Sea Disaster,Fatal or drowning or scavenging,Y,"Canberra Times, 1/6/1950",1949.12.00.b-Wingate.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1722
4790,1931.04.27.R,1931.04.27.R,1931.04.27.R,Reported 27-Apr-1931,1931,NaN,French Southern Territories,�le Saint-Paul,Quillezic,M,"Fishing, boat capsized",Unprovoked,FATAL,Y,"Los Angeles Times, 4/27/1931",1931.04.27.R-Quillezic.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1203


We will traspas the proper information and eliminate the ROW where there was no shark involved

In [208]:
for i in data.index: 
    if data["Area"][i] == "Between Comores & Madagascar":
        data["Country"][i] = "Madagascar"
    elif data["Location"][i] == "Between St. Kitts & Nevis":
        data["Country"][i] = "St. Kitts & Nevis"
    elif data["Location"][i] == "�le Saint-Paul":
        data["Country"][i] = "Le Saint-Paul"

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [209]:
no_shark = data[(data["Location"].notnull()) & (data["Area"].notnull()) & (data["Country"].isnull())]
drop_rows(no_shark)

Double check how many NULLs `Country` has to make sure the function works.

In [210]:
data["Country"].isnull().sum()

15

In [211]:
data.shape

(5952, 19)

Given that we have transfered all the needed information to `Country` we will drop the other two columns: `Area` & `Location`.

In [212]:
area_locat = ["Area", "Location"]
data = data.drop(area_locat, axis = 1)

We will check the information in the column to regularize it.

In [213]:
pd.unique(data["Country"])

array(['USA', 'AUSTRALIA', 'NEW CALEDONIA', 'REUNION', 'BAHAMAS', 'SPAIN',
       'CHINA', 'JAPAN', 'COLUMBIA', 'SOUTH AFRICA', 'EGYPT',
       'NEW ZEALAND', 'INDONESIA', 'FRENCH POLYNESIA', 'CAPE VERDE',
       'Fiji', 'BRAZIL', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS',
       'UNITED ARAB EMIRATES', 'ARUBA', 'MOZAMBIQUE', 'THAILAND', 'FIJI',
       'PUERTO RICO', 'ITALY', 'MEXICO', 'ATLANTIC OCEAN', 'GREECE',
       'MAURITIUS', 'ST. MARTIN', 'FRANCE', 'ECUADOR', 'PAPUA NEW GUINEA',
       'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA',
       'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM',
       'SEYCHELLES', 'BELIZE', 'PHILIPPINES', 'NIGERIA', 'TONGA',
       'SCOTLAND', 'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE',
       'ANTIGUA', 'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'COSTA RICA',
       'UNITED KINGDOM', 'MALAYSIA', 'UNITED ARAB EMIRATES (UAE)',
       'SAMOA', 'AZORES', 'SOLOMON ISLANDS', 'SOUTH KOREA', 'MALTA',
       'VIETNAM', 'MADAGASCAR', 'PANAMA', 'SOM

In [214]:
countries = []
for x in data["Country"]:
    z = []
    if x == "USA":
        countries.append(x)
    else:
        x = str(x).strip().title()
        countries.append(x)
data["Country"] = countries

In [215]:
filter_by_nan = data[data["Country"] == "Nan"]
drop_rows(filter_by_nan)

In [216]:
pd.unique(data["Country"])

array(['USA', 'Australia', 'New Caledonia', 'Reunion', 'Bahamas', 'Spain',
       'China', 'Japan', 'Columbia', 'South Africa', 'Egypt',
       'New Zealand', 'Indonesia', 'French Polynesia', 'Cape Verde',
       'Fiji', 'Brazil', 'Dominican Republic', 'Cayman Islands',
       'United Arab Emirates', 'Aruba', 'Mozambique', 'Thailand',
       'Puerto Rico', 'Italy', 'Mexico', 'Atlantic Ocean', 'Greece',
       'Mauritius', 'St. Martin', 'France', 'Ecuador', 'Papua New Guinea',
       'Trinidad & Tobago', 'Kiribati', 'Israel', 'Diego Garcia',
       'Taiwan', 'Jamaica', 'Palestinian Territories', 'Guam',
       'Seychelles', 'Belize', 'Philippines', 'Nigeria', 'Tonga',
       'Scotland', 'Canada', 'Croatia', 'Saudi Arabia', 'Chile',
       'Antigua', 'Kenya', 'Russia', 'Turks & Caicos', 'Costa Rica',
       'United Kingdom', 'Malaysia', 'United Arab Emirates (Uae)',
       'Samoa', 'Azores', 'Solomon Islands', 'South Korea', 'Malta',
       'Vietnam', 'Madagascar', 'Panama', 'Somalia', '

### GROUP 2: Fatal & Injury

We will focus on `Fatal` so we will be droping all of those ROW where we can't exctract information `Injury`. <br>
<br>To double check the information we will start by checking how many NULLs `Fatal` has.

In [217]:
data["Fatal"].isnull().sum()

18

Using the function we will DROP all the ROWS where `Fatal` and `Injury` are NULL

In [218]:
data[(data["Injury"].isnull()) & (data["Fatal"].isnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order


In [219]:
filter_by_fat_inj = data[(data["Injury"].isnull()) & (data["Fatal"].isnull())]
drop_rows(filter_by_fat_inj)

Double check how many NULLs `Fatal` has to make sure the function works, we will be doing this every time we pass the function.

In [220]:
data["Fatal"].isnull().sum()

18

FILTER by `Fatal`: we will check the ROWs where `Fatal` is not NULL.

In [221]:
data[(data["Injury"].isnull()) & (data["Fatal"].notnull())]

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
1047,2008.04.20.a,2008.04.20.a,2008.04.20.a,20-Apr-08,2008,Australia,Jamie Adlington,M,NaN,Unprovoked,NaN,UNKNOWN,"T. Peake, GSAF",2008.04.20.a-Adlington.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4946
1142,2007.07.00,2007.07.00,2007.07.00,Jul-07,2007,Senegal,Alex Takyi,NaN,Murder,Invalid,NaN,UNKNOWN,"Daily Guide, 8/20/2007",2007.07.00-Takyi.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4851
2025,1997.06.07,1997.06.07,1997.06.07,07-Jun-97,1997,Brazil,Jos� Luiz Lipiani,M,Bathing,Unprovoked,NaN,UNKNOWN,"Globo, 6/9/1997",1997.06.07-NV-Lipiani.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3968
2042,1996.12.29,1996.12.29,1996.12.29,29-Dec-96,1996,Australia,Blair Hall,M,Surfing,Unprovoked,NaN,UNKNOWN,"The Advertiser, 12/30/1996, p.3",1996.12.29-BlairHall.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3951
2423,1990.00.00,1990.00.00,1990.00.00,1990,1990,USA,male,M,Surfing,Unprovoked,NaN,UNKNOWN,NaN,1990.00.00-NV-Pensacola.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3571
2813,1981.03.00,1981.03.00,1981.03.00,Mar-81,1981,Brazil,NaN,NaN,Diving,Unprovoked,NaN,UNKNOWN,"Globo,",1981.03.00-Brazil.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3180
2982,1975.10.04,1975.10.04,1975.10.04,04-Oct-75,1975,USA,NaN,NaN,Surfing,Invalid,NaN,UNKNOWN,Unconfirmed Report,1975.10.04-NV-California.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3011
3073,1973.08.27,1973.08.27,1973.08.27,27-Aug-73,1973,Australia,G. Cole,NaN,NaN,Unprovoked,NaN,UNKNOWN,"J. Green, p.36",1973.08.27-Cole.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2920
3209,1969.05.22,1969.05.22,1969.05.22,22-May-69,1969,Dominican Republic,"Douglas Kuchn, Jr.",M,Surfing,Unprovoked,NaN,UNKNOWN,H.D.Baldridge (1994) SAF Case #1607,1969.05.22-NV-Kuchn.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2784
3231,1968.09.22,1968.09.22,1968.09.22,22-Sep-68,1968,USA,NaN,NaN,Surfing,Invalid,NaN,UNKNOWN,M. Vorenberg,1968.09.22-NV-RivieraBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2762


We see that all the `Fatal` that are NOT NULL = UNKNOWN the `Injury` is NaN; we will DROP this ROWS

In [222]:
filter_by_fatal = data[(data["Injury"].isnull()) & (data["Fatal"].isnull())]
drop_rows(filter_by_fatal)

In [223]:
data["Fatal"].isnull().sum()

18

FILTER by `Injury`: we will check the ROWs where `Injury` is not NULL, and see if we can complement `Fatal`.

In [224]:
(data[(data["Injury"].notnull()) & (data["Fatal"].isnull())])

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
54,2016.06.02.b,2016.06.02.b,2016.06.02.b,02-Jun-16,2016,Australia,Waade Madigan and Dr Seyong Kim,M,Spearfishing,Unprovoked,"No injury, but sharks repeatedly hit their fin...",NaN,"Gold Coast Bulletin, 6/4/2016",2016.06.02.b-Matigan.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5939
1844,2000.03.03.R,2000.03.03.R,2000.03.03.R,Reported 03-Mar-2000,2000,New Zealand,Ricky Stringer,M,Kayaking,Invalid,Reported as shark attack but probable drowning,NaN,"R. D. Weeks, GSAF",2000.03.03.R-Stringer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4149
2449,1989.07.27,1989.07.27,1989.07.27,27-Jul-89,1969,Bermuda,Russian male,M,Scuba diving,Invalid,FATAL,NaN,"LA Times, 7/28/1989",1989.07.27-SovietDiver.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3545
3280,1967.09.13,1967.09.13,1967.09.13,13-Sep-67,1967,Italy,Romeo Guarini,M,Scuba diving,Provoked,"Diver shot the shark, then it injured his arm ...",NaN,C. Moore. GSAF,1967.09.13-Guarini.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2713
3901,1958.12.28,1958.12.28,1958.12.28,28-Dec-58,1958,Turkey,Fishing boat. Occupants: Yunus Potur & Ali Durmaz,NaN,Fishing,Boat,Boat damaged,NaN,"C. Moore, GSAF",1958.12.28-Constantinople.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2092
4107,1954.07.27,1954.07.27,1954.07.27,27-Jul-54,1954,Italy,10 crew,M,Fishing trawler Flavio Gioia,Boating,No injury to occupants. Shark tore nets & traw...,NaN,"C. Moore, GSAF",1954.07.27-Trawler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1886
4112,1954.07.01.R,1954.07.01.R,1954.07.01.R,Reported 01-Jul-1954,1954,Croatia,male,NaN,NaN,Invalid,Human remains found in shark,NaN,"C. Moore, GSAF",1954.07.01.R-Pula.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1881
5307,1901.07.17,1901.07.17,1901.07.17,17-Jul-01,1901,Italy,Antonio Tornatori,M,Swimming,Invalid,"Disappeared, but shark involvement unconfirmed",NaN,C. Moore,1901.07.17-Antonio.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,686
5437,1892.06.24,1892.06.24,1892.06.24,24-Jun-1892,1892,France,NaN,NaN,Fishing boat,Invalid,"No injury, no attack",NaN,"C.Moore, GSAF",1892.06.24-Brest.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,556
5468,1889.11.29.R,1889.11.29.R,1889.11.29.R,Reported 29-Nov-1889,1889,Greece,NaN,NaN,NaN,Invalid,"Human remains found in 4m, 900 kg shark",NaN,"C. Moore, GSAF",1889.11.29.R-Greece.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,525


Seen how we can fill in `Fatal` with `Injury` we will complement it.

We can see we have free text and tabulations so we will use two different methods:

**Method 1**

In [225]:
#for tabulations use a loop
for i in data.index: 
    if data["Injury"][i] == "FATAL":
        data["Fatal"][i] = "Y"

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


**Method 2**

In [226]:
#for free writting use string operations
data.loc[data["Injury"].str.contains("FATAL.", na = False), "Fatal"] = "Y"
data.loc[data["Injury"].str.contains("remains", na = False), "Fatal"] = "Y"
data.loc[data["Injury"].str.contains("drowning", na = False), "Fatal"] = "Y"
data.loc[data["Injury"].str.contains("Death", na = False), "Fatal"] = "Y"
data.loc[data["Injury"].str.contains("Disappeared", na = False), "Fatal"] = "Y"
data.loc[data["Injury"].str.contains("No injury", na = False), "Fatal"] = "N"
data.loc[data["Injury"].str.contains("no injury", na = False), "Fatal"] = "N"
data.loc[data["Injury"].str.contains("Boat", na = False), "Fatal"] = "N"
data.loc[data["Injury"].str.contains("injured", na = False), "Fatal"] = "N"


Seen that we have filled the corresponding cells in `Fatal` with the ones in `Injury` we will DROP the second column

In [258]:
data = data.drop("Injury", axis = 1)
data

,Year,Country,Sex,Activity,Type,Fatal
0,2016,USA,M,Surfing,Unprovoked,N
1,2016,USA,M,Surfing,Unprovoked,N
2,2016,USA,M,Surfing,Unprovoked,N
3,2016,Australia,M,Surfing,Unprovoked,N
4,2016,Australia,M,Surfing,Unprovoked,N
...,...,...,...,...,...,...
5987,Before 1903,Australia,M,Diving,Unprovoked,Y
5988,Before 1903,Australia,M,Pearl diving,Unprovoked,Y
5989,1900-1905,USA,M,Swimming,Unprovoked,Y
5990,1883-1889,Panama,M,NaN,Unprovoked,Y


So far we have made quite a bit of changes so we will check the shape once more

In [228]:
data.shape

(5937, 17)

### GROUP 3: SEX

For `Sex`, we will do something a bit different, given that it is only one column and it is more if a tabulation, we will do the same with `Type`.

We will start the same, checking for NULLs

In [229]:
data["Sex"].isnull().sum()

564

We will check all the unique values that we have

In [230]:
data["Sex"].value_counts()

M      4788
F       580
M         2
lli       1
N         1
.         1
Name: Sex, dtype: int64

Using logic we will replace the ones we can with the correct answer, the ones we don't know we will categorize them as Unkown

In [231]:
data['Sex'] = data['Sex'].str.replace('M ', 'M') #extra space
data['Sex'] = data['Sex'].str.replace('N', 'M') # N next to the M
data['Sex'] = data['Sex'].str.replace('.', 'M') # . next to the M
data['Sex'] = data['Sex'].str.replace('lli', 'Unkown') # lli unknown meaning
data['Sex'] = data['Sex'].fillna('Unkown')

We will double check the values and the NULLs

In [232]:
set(data["Sex"])
data["Sex"].isnull().sum()

{'F', 'M', 'Unkown'}

0

### GROUP 4: TYPE

We will follow a very similar path with what we did in `Sex`.

We will start the same, checking for NULLs

In [233]:
data["Type"].isnull().sum()

0

In [234]:
data["Type"].value_counts()

Unprovoked      4350
Provoked         552
Invalid          512
Sea Disaster     216
Boat             199
Boating          108
Name: Type, dtype: int64

We unify the similar ones (*Boating* and *Boat*)

In [235]:
data["Type"] = data["Type"].str.replace("Boating", "Boat")

We will double check the values, in this case, given that we have 0 NULLs it's not needed to check it.

In [236]:
set(data["Type"])

{'Boat', 'Invalid', 'Provoked', 'Sea Disaster', 'Unprovoked'}

### GROUP 5: DATE & YEAR

`Date` and `Year` are quite difficult coulumns because strigs and integers are mixed.

From the begining we saw that both columns has no NULLs so we don't have to revise that.

In [238]:
(data["Year"] == 0).sum()
(data["Year"] == str).sum()

115

0

In [239]:
pd.DataFrame(data,columns=['Year','Date']).tail()

,Year,Date
5987,0,Before 1903
5988,0,Before 1903
5989,0,1900-1905
5990,0,1883-1889
5991,0,1845-1853


In [240]:
#starting from the bottom up most if the years are 0 and the dates are years so we will organize the info 
#in the proper columns
for i in data.index: #pq accederemos a 2 columnas no solo a una
    if data["Year"][i] == 0:
        data["Year"][i] = data["Date"][i]
data = data

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Now we have at least 124 repeated pieces of information given that we've copied from `Date` to `Year`.

We will have to check what information has been transfered and if there's any free wrting in the years.

After, given that we want to extract the month from `Date`, if the information is the exact same as in `Year` we will know that the months will not be reflected and we can assign them a new value: *Unkown*


In [242]:
def clean_dates(data):
    for i in data.index: 
        if data["Date"][i] == data["Year"][i]:
             data["Date"][i] = "Unknown"
        elif str(data["Year"][i]).startswith("[a-zA-Z]+ [a-zA-Z]+"):
            data["Year"][i] = data["Year"][i][-4:]
        #elif str(data["Year"][i]).endswith("\"):
            #data["Year"][i] = data["Year"][i][4:]
        elif "war" in str(data["Year"][i]) or "War" in str(data["Year"][i]) :
            data["Year"][i] = "1945"
        elif data["Year"][i] == "No date" or len(str(data["Year"][i])) < 4:
            data["Year"][i] = "Unkown"
clean_dates(data)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [243]:
data

,Case Number,Case Number 1,Case Number 2,Date,Year,Country,Name,Sex,Activity,Type,Injury,Fatal,Investigator or Source,Pdf,Href Formula,Href,Original Order
0,2016.09.18.c,2016.09.18.c,2016.09.18.c,18-Sep-16,2016,USA,male,M,Surfing,Unprovoked,Minor injury to thigh,N,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,2016.09.18.b,2016.09.18.b,18-Sep-16,2016,USA,Chucky Luciano,M,Surfing,Unprovoked,Lacerations to hands,N,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,2016.09.18.a,2016.09.18.a,18-Sep-16,2016,USA,male,M,Surfing,Unprovoked,Lacerations to lower leg,N,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,2016.09.17,2016.09.17,17-Sep-16,2016,Australia,Rory Angiolella,M,Surfing,Unprovoked,Struck by fin on chest & leg,N,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,2016.09.16,2016.09.15,16-Sep-16,2016,Australia,male,M,Surfing,Unprovoked,No injury: Knocked off board by shark,N,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,ND.0005,ND.0005,Unknown,Before 1903,Australia,male,M,Diving,Unprovoked,FATAL,Y,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6
5988,ND.0004,ND.0004,ND.0004,Unknown,Before 1903,Australia,Ahmun,M,Pearl diving,Unprovoked,FATAL,Y,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5
5989,ND.0003,ND.0003,ND.0003,Unknown,1900-1905,USA,Coast Guard personnel,M,Swimming,Unprovoked,FATAL,Y,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,4
5990,ND.0002,ND.0002,ND.0002,Unknown,1883-1889,Panama,Jules Patterson,M,NaN,Unprovoked,FATAL,Y,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3


# Elimination

Once we have studied and organized the values and columns of the table that we know (or think) will be usesful we can go ahead an eliminate those columns that we will not use.

If we print `Case Number`, `Case Number 1`, `Case Number 2` and `Date` we see it's quite similar so we won't be needing the first three columns.


In [142]:
data[data["Case Number"].notnull() & data["Case Number 1"].notnull() & data["Case Number 2"].notnull() & data["Date"].notnull()].head()

KeyError: 'Case Number'

We DROP `Case Number`, `Case Number 1`, `Case Number 2` and pass the colum list to make sure it's been properly deleted.

In [250]:
dupl = ['Case Number', 'Case Number 1', 'Case Number 2',]
data = data.drop(dupl, axis = 1)

In [251]:
data.columns

Index(['Year', 'Country', 'Name', 'Sex', 'Activity', 'Type', 'Injury', 'Fatal',
       'Investigator or Source', 'Pdf', 'Href Formula', 'Href',
       'Original Order'],
      dtype='object')

From 'Href Formula', 'Href', 'Name', 'Investigator or Source', 'PDF' and 'Original Order' we won't be needing any information in this case, so we will go ahead and delete them.

In [252]:
unnecs = ['Name','Href Formula', 'Href','Investigator or Source', 'Pdf', 'Original Order']
data = data.drop(unnecs, axis = 1)

And lets not forget to pass the colum list to make sure it's been properly deleted.

In [253]:
data.columns

Index(['Year', 'Country', 'Sex', 'Activity', 'Type', 'Injury', 'Fatal'], dtype='object')

The following columns are being deleted because I've had not had time to finish them so basically they ugly so i'm droping them

In [248]:
data = data.drop("Date", axis = 1)

# Final DataSet and Extraction

Looking at out final product

In [259]:
data

,Year,Country,Sex,Activity,Type,Fatal
0,2016,USA,M,Surfing,Unprovoked,N
1,2016,USA,M,Surfing,Unprovoked,N
2,2016,USA,M,Surfing,Unprovoked,N
3,2016,Australia,M,Surfing,Unprovoked,N
4,2016,Australia,M,Surfing,Unprovoked,N
...,...,...,...,...,...,...
5987,Before 1903,Australia,M,Diving,Unprovoked,Y
5988,Before 1903,Australia,M,Pearl diving,Unprovoked,Y
5989,1900-1905,USA,M,Swimming,Unprovoked,Y
5990,1883-1889,Panama,M,NaN,Unprovoked,Y


Checking the NULLS of our final table

In [263]:
data.isnull().sum()* 100 / len(data)

Year        0.000000
Country     0.000000
Sex         0.000000
Activity    8.741789
Type        0.000000
Fatal       0.000000
dtype: float64

Last size check of the table

In [264]:
data.shape

(5937, 6)

Document creation

In [266]:
data.to_csv("./shark_attack.csv", index = False)

***AYUDA***